In [1]:
import torch
import os
import pickle
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
class LexicalAcousticVideoClassifier(torch.nn.Module):
    def __init__(self, embedding_size):
        super(LexicalAcousticVideoClassifier, self).__init__()
        self.linear1 = torch.nn.Linear(embedding_size, 16)
        self.linear2 = torch.nn.Linear(16, 2)
        torch.nn.init.kaiming_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
    
    def forward(self, x):
        x = self.linear1(torch.nn.functional.relu(x))
        return self.linear2(torch.nn.functional.relu(x))

In [3]:
class AudioClassifierPCA(torch.nn.Module):
    def __init__(self, embedding_size):
        super(AudioClassifierPCA, self).__init__()
        self.linear1 = torch.nn.Linear(embedding_size, 16)
        self.linear2 = torch.nn.Linear(16, 2)
        # Xavier initialize the linear layer
        torch.nn.init.kaiming_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
    
    def forward(self, x):
        x = self.linear1(torch.nn.functional.relu(x))
        return self.linear2(torch.nn.functional.relu(x))

In [4]:
class LexicalBertClassifierPCA(torch.nn.Module):
    def __init__(self, embedding_size):
        super(LexicalBertClassifierPCA, self).__init__()
        self.linear1 = torch.nn.Linear(embedding_size, 16)
        self.linear2 = torch.nn.Linear(16, 2)
        torch.nn.init.kaiming_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
    
    def forward(self, x):
        x = self.linear1(torch.nn.functional.relu(x))
        return self.linear2(torch.nn.functional.relu(x))

In [5]:
class VisualClassifierPCA(torch.nn.Module):
    def __init__(self, embedding_size):
        super(VisualClassifierPCA, self).__init__()
        self.linear1 = torch.nn.Linear(embedding_size, 16)
        self.linear2 = torch.nn.Linear(16, 2)
        # Xavier initialize the linear layer
        torch.nn.init.kaiming_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
    
    def forward(self, x):
        x = self.linear1(torch.nn.functional.relu(x))
        return self.linear2(torch.nn.functional.relu(x))

In [6]:
class LexicalAcousticVideoLateFusion(torch.nn.Module):
    def __init__(self, embedding_size):
        super(LexicalAcousticVideoLateFusion, self).__init__()
        self.acoustic = AudioClassifierPCA(embedding_size // 3)
        self.lexical = LexicalBertClassifierPCA(embedding_size // 3)
        self.visual = VisualClassifierPCA(embedding_size // 3)
    
    def forward(self, x):
        # Split x into 3 parts
        x = torch.split(x, x.shape[1] // 3, dim=1)
        return (self.lexical(x[0]), self.acoustic(x[1]), self.visual(x[2]))

In [7]:
class LexicalAcousticVideoDataset(Dataset):
    def __init__(self, data, labels, files, pca, test):
        lexical_data, acoustic_data, video_data = data
        lexical_file, acoustic_file, video_file = files
        pca_l, sc_l, pca_a, sc_a, pca_v, sc_v = pca
        self.lexical_embeddings = []
        with open(lexical_file, 'rb') as f:
            self.embedding_map = pickle.load(f)
        for file in lexical_data:
            embedding = self.embedding_map[file.split('.')[0]]
            if type(embedding) == np.ndarray:
                self.lexical_embeddings.append(embedding)
            else:
                self.lexical_embeddings.append(embedding.numpy())
        self.lexical_embeddings = np.array(self.lexical_embeddings)
        with open(video_file, 'rb') as f:
            embeddings_map = pickle.load(f)
        self.video_embeddings = np.array([embeddings_map[file.split('.')[0]].numpy() for file in video_data])
        with open(acoustic_file, 'rb') as f:
            embeddings_map = pickle.load(f)
        self.acoustic_embeddings = np.array([embeddings_map[file] for file in acoustic_data])
        if not test:
            self.lexical_embeddings = sc_l.fit_transform(self.lexical_embeddings)
            self.lexical_embeddings = pca_l.fit_transform(self.lexical_embeddings)
            self.acoustic_embeddings = sc_a.fit_transform(self.acoustic_embeddings)
            self.acoustic_embeddings = pca_a.fit_transform(self.acoustic_embeddings)
            self.video_embeddings = sc_v.fit_transform(self.video_embeddings)
            self.video_embeddings = pca_v.fit_transform(self.video_embeddings)
        else:
            self.lexical_embeddings = sc_l.transform(self.lexical_embeddings)
            self.lexical_embeddings = pca_l.transform(self.lexical_embeddings)
            self.acoustic_embeddings = sc_a.transform(self.acoustic_embeddings)
            self.acoustic_embeddings = pca_a.transform(self.acoustic_embeddings)
            self.video_embeddings = sc_v.transform(self.video_embeddings)
            self.video_embeddings = pca_v.transform(self.video_embeddings)
        self.lexical_embeddings = torch.from_numpy(self.lexical_embeddings).float()
        self.acoustic_embeddings = torch.from_numpy(self.acoustic_embeddings).float()
        self.video_embeddings = torch.from_numpy(self.video_embeddings).float()
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return torch.concat((self.lexical_embeddings[index], self.acoustic_embeddings[index], self.video_embeddings[index])), self.labels[index]

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
def train(model, train_loader, criterion, optimizer, num_epochs, late_fusion):
    for _ in range(num_epochs):
        model.train()
        for x, y in train_loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            if late_fusion:
                loss = criterion(out[0], y) + criterion(out[1], y) + criterion(out[2], y)
            else:
                loss = criterion(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [10]:
def eval(model, val_loader, late_fusion):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            if not late_fusion:
                _, predicted = torch.max(out.data, 1)
            else:
                _, p0 = torch.max(out[0].data, 1)
                _, p1 = torch.max(out[1].data, 1)
                _, p2 = torch.max(out[2].data, 1)
                predicted = p0
                predicted[p1 == p2] = p1[p1 == p2]
            total += y.size(0)
            correct += (predicted == y).sum().item()
        return correct / total

In [11]:
def get_lexical_data(TRUTH_DIR, DECEPTION_DIR):
    data = []
    for file in os.listdir(TRUTH_DIR):
        data.append(file)
    for file in os.listdir(DECEPTION_DIR):
        data.append(file)
    return data

In [31]:
train_acoustic_data = []
train_labels = []
for file in os.listdir('../../data/Acoustic/Truthful/'):
    train_acoustic_data.append(file)
    train_labels.append(0)
for file in os.listdir('../../data/Acoustic/Deceptive/'):
    train_acoustic_data.append(file)
    train_labels.append(1)

test_acoustic_data = []
test_labels = []
for file in os.listdir('../../Dataset2/Audio/Truth/'):
    key = file.split('.')[0]
    key = '-'.join(key.split('-')[:3])
    test_acoustic_data.append(key)
    test_labels.append(0)
for file in os.listdir('../../Dataset2/Audio/Lies/'):
    key = file.split('.')[0]
    key = '-'.join(key.split('-')[:3])
    test_acoustic_data.append(key)
    test_labels.append(1)

train_video_data = []
for file in os.listdir('../../data/Clips/Truthful/'):
    train_video_data.append(file.split('.')[0])
for file in os.listdir('../../data/Clips/Deceptive/'):
    train_video_data.append(file.split('.')[0])

test_video_data = []
for file in os.listdir('../../Dataset2/Videos/Truth/'):
    key = file.split('.')[0]
    key = '-'.join(key.split('-')[:3])
    test_video_data.append(key)
for file in os.listdir('../../Dataset2/Videos/Lie/'):
    key = file.split('.')[0]
    key = '-'.join(key.split('-')[:3])
    test_video_data.append(key)

train_lexical_data = get_lexical_data('../../data/Transcription/Truthful/', '../../data/Transcription/Deceptive/')
test_lexical_data = get_lexical_data('../../Dataset2/Text/Truth/', '../../Dataset2/Text/Lie/')

In [32]:
def get_accuracy(train_lexical_file, train_acoustic_file, train_video_file, test_lexical_file, test_acoustic_file, test_video_file, embedding_size, train_batch_size, num_epochs, late_fusion):
    sc_a = StandardScaler()
    pca_a = PCA(n_components=embedding_size // 3)
    sc_v = StandardScaler()
    pca_v = PCA(n_components=embedding_size // 3)
    sc_l = StandardScaler()
    pca_l = PCA(n_components=embedding_size // 3)
    train_dataset = LexicalAcousticVideoDataset((train_lexical_data, train_acoustic_data, train_video_data), train_labels, (train_lexical_file, train_acoustic_file, train_video_file), (pca_l, sc_l, pca_a, sc_a, pca_v, sc_v), False)
    test_dataset = LexicalAcousticVideoDataset((test_lexical_data, test_acoustic_data, test_video_data), test_labels, (test_lexical_file, test_acoustic_file, test_video_file), (pca_l, sc_l, pca_a, sc_a, pca_v, sc_v), True)
    if late_fusion:
        model = LexicalAcousticVideoLateFusion(embedding_size).to(device)
    else:
        model = LexicalAcousticVideoClassifier(embedding_size).to(device)
    train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    learning_rate = 1e-3
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train(model, train_loader, criterion, optimizer, num_epochs, late_fusion)
    return eval(model, test_loader, late_fusion)

In [33]:
# Model accuracy for bert-base-uncased, vggish and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_1.pkl', '../../embeddings/acoustic_features_2.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_1.pkl', '../../embeddings2/acoustic_features_2.pkl', '../../embeddings2/visual_features.pkl', 96, 4, 20, False)

0.4697986577181208

In [34]:
# Model accuracy for bert-base-cased-finetuned-emotion and vggish and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_2.pkl', '../../embeddings/acoustic_features_2.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_2.pkl', '../../embeddings2/acoustic_features_2.pkl', '../../embeddings2/visual_features.pkl', 96, 4, 20, False)

0.5033557046979866

In [35]:
# Model accuracy for transformersbook/distilbert-base-uncased-finetuned-emotion and vggish and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_3.pkl', '../../embeddings/acoustic_features_2.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_3.pkl', '../../embeddings2/acoustic_features_2.pkl', '../../embeddings2/visual_features.pkl', 96, 4, 20, False)

0.47651006711409394

In [36]:
# Model accuracy for mpNet and vggish and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_6.pkl', '../../embeddings/acoustic_features_2.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_6.pkl', '../../embeddings2/acoustic_features_2.pkl', '../../embeddings2/visual_features.pkl', 96, 4, 20, False)

0.5033557046979866

In [37]:
# Model accuracy for bert-base-uncased, vggish and resnet-3d-18 Late Fusion
get_accuracy('../../embeddings/transcript_features_1.pkl', '../../embeddings/acoustic_features_2.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_1.pkl', '../../embeddings2/acoustic_features_2.pkl', '../../embeddings2/visual_features.pkl', 96, 4, 20, True)

0.48322147651006714

In [38]:
# Model accuracy for bert-base-uncased, vggish and resnet-3d-18 Late Fusion
get_accuracy('../../embeddings/transcript_features_1.pkl', '../../embeddings/acoustic_features_2.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_1.pkl', '../../embeddings2/acoustic_features_2.pkl', '../../embeddings2/visual_features.pkl', 192, 4, 20, True)

0.4697986577181208

In [39]:
# Model accuracy for bert-base-cased-finetuned-emotion and vggish and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_2.pkl', '../../embeddings/acoustic_features_2.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_2.pkl', '../../embeddings2/acoustic_features_2.pkl', '../../embeddings2/visual_features.pkl', 96, 4, 20, True)

0.48322147651006714

In [40]:
# Model accuracy for bert-base-cased-finetuned-emotion and vggish and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_2.pkl', '../../embeddings/acoustic_features_2.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_2.pkl', '../../embeddings2/acoustic_features_2.pkl', '../../embeddings2/visual_features.pkl', 192, 4, 20, True)

0.46308724832214765

In [41]:
# Model accuracy for mpNet and vggish and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_6.pkl', '../../embeddings/acoustic_features_2.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_6.pkl', '../../embeddings2/acoustic_features_2.pkl', '../../embeddings2/visual_features.pkl', 96, 4, 20, True)

0.46308724832214765

In [42]:
# Model accuracy for mpNet and vggish and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_6.pkl', '../../embeddings/acoustic_features_2.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_6.pkl', '../../embeddings2/acoustic_features_2.pkl', '../../embeddings2/visual_features.pkl', 192, 4, 20, True)

0.5570469798657718